# UIDAI Biometric Data Analysis file 1

In [ ]:
#importing necessary libraries
import pandas as pd

In [ ]:
df = pd.read_csv('/content/bio_cleaned_final.csv')

In [ ]:
df.shape

(1408318, 7)

In [ ]:
df.head()

,Unnamed: 0,date,state,district,pincode,bio_age_5_17,bio_age_17_
0,0,01-03-2025,haryana,mahendragarh,123029,280,577
1,1,01-03-2025,bihar,madhepura,852121,144,369
2,2,01-03-2025,bihar,bhojpur,802158,256,980
3,3,01-03-2025,tamil nadu,madurai,625514,271,815
4,4,01-03-2025,maharashtra,ratnagiri,416702,155,529


In [ ]:
df.drop(['Unnamed: 0'],inplace=True,axis=1)

In [ ]:
df['district'] = df['district'].str.upper()
df['state'] = df['state'].str.upper()

In [ ]:
df.head()

,date,state,district,pincode,bio_age_5_17,bio_age_17_
0,01-03-2025,HARYANA,MAHENDRAGARH,123029,280,577
1,01-03-2025,BIHAR,MADHEPURA,852121,144,369
2,01-03-2025,BIHAR,BHOJPUR,802158,256,980
3,01-03-2025,TAMIL NADU,MADURAI,625514,271,815
4,01-03-2025,MAHARASHTRA,RATNAGIRI,416702,155,529


In [ ]:
df['state']=df['state'].str.lower()
df['district']=df['district'].str.lower()

In [ ]:
df.head()

,date,state,district,pincode,bio_age_5_17,bio_age_17_
0,01-03-2025,haryana,mahendragarh,123029,280,577
1,01-03-2025,bihar,madhepura,852121,144,369
2,01-03-2025,bihar,bhojpur,802158,256,980
3,01-03-2025,tamil nadu,madurai,625514,271,815
4,01-03-2025,maharashtra,ratnagiri,416702,155,529


In [ ]:
df.shape

(1408318, 6)

In [ ]:
df_pre_final = df.copy()

In [ ]:
df_pre_final.shape

(1408318, 6)

In [ ]:
df_pre_final.columns

Index(['date', 'state', 'district', 'pincode', 'bio_age_5_17', 'bio_age_17_'], dtype='object')

In [ ]:
import pandas as pd
import json

# Loading pincode JSON file
with open('pin-code-data.json', 'r') as f:
    pincode_data = json.load(f)

# Create a mapping of state to its valid pincodes
state_to_pincodes = {}
for pincode, info in pincode_data.items():
    state = info['state'].upper()

    if state not in state_to_pincodes:
        state_to_pincodes[state] = set()
    state_to_pincodes[state].add(pincode)

# Function to check if pincode belongs to state
def check_pincode_in_state(pincode, state):
    """
    Check if pincode belongs to the given state
    Returns: True if pincode exists for that state, False otherwise
    """
    pincode_str = str(pincode).strip()
    state_upper = state.upper().strip()

    if state_upper in state_to_pincodes:
        return pincode_str in state_to_pincodes[state_upper]
    else:
        return False

# Create new 'match' column

df_pre_final['match'] = df_pre_final.apply(
    lambda row: check_pincode_in_state(row['pincode'], row['state']),
    axis=1
)

# Display results
print(df_pre_final[['state', 'pincode', 'match']])

# Get summary
print("\nMatch Summary:")
print(df_pre_final['match'].value_counts())

# Show unmatched records
unmatched = df_pre_final[df_pre_final['match'] == False]
print(f"\nUnmatched Records: {len(unmatched)}")
print(unmatched[['state', 'pincode', 'match']])


               state  pincode  match
0            haryana   123029   True
1              bihar   852121   True
2              bihar   802158   True
3         tamil nadu   625514   True
4        maharashtra   416702   True
...              ...      ...    ...
1408313  west bengal   743384  False
1408314  west bengal   743392  False
1408315  west bengal   743502   True
1408316  west bengal   733201   True
1408317  west bengal   733213  False

[1408318 rows x 3 columns]

Match Summary:
match
True     1359107
False      49211
Name: count, dtype: int64

Unmatched Records: 49211
                 state  pincode  match
19       uttar pradesh   225408  False
80               bihar   852210  False
136              bihar   800018  False
169        west bengal   733252  False
179      uttar pradesh   241501  False
...                ...      ...    ...
1408298    west bengal   721665  False
1408299    west bengal   721669  False
1408313    west bengal   743384  False
1408314    west bengal   74339

In [ ]:
df_clean_after_state_check = df_pre_final[df_pre_final['match']==True]

In [ ]:
df_clean_after_state_check.shape

(1359107, 7)

In [ ]:
df_clean_after_state_check.head()

,date,state,district,pincode,bio_age_5_17,bio_age_17_,match
0,01-03-2025,haryana,mahendragarh,123029,280,577,True
1,01-03-2025,bihar,madhepura,852121,144,369,True
2,01-03-2025,bihar,bhojpur,802158,256,980,True
3,01-03-2025,tamil nadu,madurai,625514,271,815,True
4,01-03-2025,maharashtra,ratnagiri,416702,155,529,True


In [ ]:
df_clean_after_state_check.to_csv('bio_final_cleaned_state_check.csv',index=False)

# Now Saving False data, where state not mapped to correct pincode

In [ ]:
df_false_after_state_check = df_clean_after_state_check = df_pre_final[df_pre_final['match']==False]

In [ ]:
df_false_after_state_check.shape

(49211, 7)

In [ ]:
df_false_after_state_check.to_csv('bio_false_after_state_check.csv',index=False)